<a href="https://colab.research.google.com/github/igorafaelms/deeplearningddi/blob/master/AI_integration2PACS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pynetdicom

     |████████████████████████████████| 1.5MB 2.7MB/s 
     |████████████████████████████████| 35.3MB 120kB/s 


In [0]:
import pynetdicom
import pydicom
from pynetdicom import AE
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelFind, \
StudyRootQueryRetrieveInformationModelFind

In [21]:
addr = "www.dicomserver.co.uk"
port = 104


ds = pydicom.Dataset()
ds.PatientName = '*TEST*'
ds.QueryRetrieveLevel = 'PATIENT'

ae = AE(ae_title=b'MY_ECHO_SCU')
# Verification SOP Class has a UID of 1.2.840.10008.1.1
#   we can use the UID str directly when adding the requested
#   presentation context
ae.add_requested_context('1.2.840.10008.1.1')
ae.add_requested_context(StudyRootQueryRetrieveInformationModelFind)
ae.add_requested_context(PatientRootQueryRetrieveInformationModelFind)

# Associate with a peer AE
assoc = ae.associate(addr, port)

if assoc.is_established:
    # Send a DIMSE C-ECHO request to the peer
    status = assoc.send_c_echo()

    # Print the response from the peer
    if status:
        print('C-ECHO Response: 0x{0:04x}'.format(status.Status))
        responses = assoc.send_c_find(ds,PatientRootQueryRetrieveInformationModelFind)
        for (status, identifier) in responses:
            if status:
                print('C-FIND query status: 0x{0:04x}'.format(status.Status))

                # If the status is 'Pending' then identifier is the C-FIND response
                if status.Status in (0xFF00, 0xFF01):
                    print(identifier)
            else:
                print('Connection timed out, was aborted or received invalid response')
    # Release the association
    assoc.release()
    print("Association Released.")

C-ECHO Response: 0x0000
C-FIND query status: 0xff00
(0008, 0005) Specific Character Set              CS: ''
(0008, 0052) Query/Retrieve Level                CS: 'PATIENT'
(0008, 0054) Retrieve AE Title                   AE: 'ANY-SCP'
(0010, 0010) Patient's Name                      PN: 'COM PDF import test'
C-FIND query status: 0xff00
(0008, 0005) Specific Character Set              CS: ''
(0008, 0052) Query/Retrieve Level                CS: 'PATIENT'
(0008, 0054) Retrieve AE Title                   AE: 'ANY-SCP'
(0010, 0010) Patient's Name                      PN: 'COM PDF import Test'
C-FIND query status: 0xff00
(0008, 0005) Specific Character Set              CS: ''
(0008, 0052) Query/Retrieve Level                CS: 'PATIENT'
(0008, 0054) Retrieve AE Title                   AE: 'ANY-SCP'
(0010, 0010) Patient's Name                      PN: 'TEST-DEMO^TEST-DEMO'
C-FIND query status: 0xff00
(0008, 0005) Specific Character Set              CS: ''
(0008, 0052) Query/Retrieve Level    

In [19]:
for _r in response:
  print(_r)

(, None)
